# Project 3: Predicting the Success of a Kickstarter Campaign
A supervised learning exercise featuring logistic regression, SVM, KNN, and [Others?]

In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from flask_sqlalchemy import SQLAlchemy
import os

In [2]:
import sys
sys.executable

'/Users/brianmcmahon/anaconda3/envs/tensorflow1.4/bin/python'

In [3]:
# env variable at tensorflow1.4 per https://conda.io/docs/user-guide/tasks/manage-environments.html#saving-environment-variables
# and https://vsupalov.com/flask-sqlalchemy-postgres/

def get_env_variable(name):
    try:
        return os.environ[name]
    except KeyError:
        message = "Expected environment variable '{}' not set.".format(name)
        raise Exception(message)

# the values of those depend on your setup
POSTGRES_URL = get_env_variable("POSTGRES_URL")
POSTGRES_USER = get_env_variable("POSTGRES_USER")
POSTGRES_PW = get_env_variable("POSTGRES_PW")
POSTGRES_DB = get_env_variable("POSTGRES_DB")

def to_str(obj):
    return str(obj).split(' ')[0]

In [4]:
DB_URL = 'postgresql+psycopg2://{user}:{pw}@{url}/{db}'.format(user=POSTGRES_USER,pw=POSTGRES_PW,url=POSTGRES_URL,db=POSTGRES_DB)

In [5]:
engine_var = DB_URL
engine = create_engine(engine_var)

In [6]:
# df = pd.read_csv('data/Kickstarter_2018-01-12T10_20_09_196Z/Kickstarter048.csv')

df = pd.read_csv('data/ks-projects-201801.csv') # From kaggle dataset at https://www.kaggle.com/kemical/kickstarter-projects
df.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


In [7]:
df['deadline'] = pd.to_datetime(df['deadline'])
df['launched'] = pd.to_datetime(df['launched']) 
df['launched'] = df['launched'].apply(lambda x: x.strftime('%Y-%m-%d'))
df['launched'] = pd.to_datetime(df['launched']) 
df = df.sort_values(['deadline'], ascending=[False])
df['campaign_length'] = (df['deadline'] - df['launched']).apply(to_str)
df['pct_goal_achieved'] = round((df['usd_pledged_real'] / df['usd_goal_real'])*100,1)
# df = df.set_index('ID')
print(df.shape)
df.head()

(378661, 17)


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,campaign_length,pct_goal_achieved
226982,22387366,Nerd Collect,Apps,Technology,GBP,2018-03-03,150000.0,2018-01-02,0.0,live,0,GB,0.0,0.0,204596.60,60,0.0
7164,1036415983,Aikyam : Onnu,Music,Music,USD,2018-03-03,10000.0,2018-01-02,174.0,live,3,US,174.0,174.0,10000.00,60,1.7
180250,1916988520,Back in Black Hills Movie,Drama,Film & Video,USD,2018-03-03,5500.0,2018-01-02,0.0,live,0,US,0.0,0.0,5500.00,60,0.0
71730,1365286494,From the Wilderness,Drama,Film & Video,USD,2018-03-02,2500.0,2018-01-01,482.0,live,19,US,0.0,482.0,2500.00,60,19.3
292034,55596200,Plateforme de statistiques,Apps,Technology,EUR,2018-03-02,9000.0,2018-01-01,0.0,live,0,FR,0.0,0.0,10923.26,60,0.0


In [8]:
# remove canceled, undefined, live and suspended
df = df[(df['state'] == 'successful') | (df['state'] == 'failed')]
df = pd.DataFrame(df)
start_date = pd.to_datetime('2012-01-01') 
df = df[df['launched'] >= start_date] # filter from start date to current
print(df.shape)
df.tail()   

(296870, 17)


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,campaign_length,pct_goal_achieved
109284,1555346070,Decentralized Dance Party- HOUSTON EDITION!,Public Art,Art,USD,2012-01-14,999.0,2012-01-10,1057.07,successful,40,US,1057.07,1057.07,999.0,4,105.8
323061,715382757,"""UPLDR"" - A Sci-Fi Short Film",Shorts,Film & Video,USD,2012-01-13,15000.0,2012-01-05,15031.00,successful,36,US,15031.00,15031.00,15000.0,8,100.2
171673,1873328240,#yougotnogame film,Shorts,Film & Video,USD,2012-01-09,500.0,2012-01-02,736.00,successful,14,US,736.00,736.00,500.0,7,147.2
361999,914600915,"""Drifting Into The Sublime"" Ken Verheecke's Ne...",World Music,Music,USD,2012-01-08,6000.0,2012-01-02,4311.00,failed,28,US,4311.00,4311.00,6000.0,6,71.9
181280,1922604394,THE LAST FIVE YEARS,Theater,Theater,USD,2012-01-07,1000.0,2012-01-03,2145.00,successful,24,US,2145.00,2145.00,1000.0,4,214.5


In [9]:
# df['ID'] = df['ID'].apply(to_str)
df['deadline'] = df['deadline'].apply(to_str)
df['launched'] = df['launched'].apply(to_str)
df['idx'] = df['ID']
df = df.set_index('idx')

In [10]:
# Prep for categorical analysis
# df.main_category = pd.Categorical(df.main_category) 
# df['main_category_code'] = df.main_category.cat.codes

# df.country = pd.Categorical(df.country) 
# df['country_code'] = df.country.cat.codes

# df.currency = pd.Categorical(df.currency) 
# df['currency_code'] = df.currency.cat.codes

# df.state = pd.Categorical(df.state) 
# df['state_code'] = df.state.cat.codes

# df.head(100)

In [11]:
# df['main_category'] = pd.get_dummies('main_category')
# df.columns

In [12]:
df_dummies = pd.get_dummies(df[['state','main_category','country','currency']],drop_first=True)
# df['state'] = pd.get_dummies(df['state'])
# df['main_category'] = pd.get_dummies(df['main_category'])
# df['country'] = pd.get_dummies(df['country'])
# df['currency'] = pd.get_dummies(df['currency'])
df['ID'].head()

idx
1031685482    1031685482
1176736360    1176736360
1894703097    1894703097
311863059      311863059
2076121120    2076121120
Name: ID, dtype: int64

In [13]:
df = df_dummies.merge(df,how='inner',left_index=True, right_index=True)

df.columns

Index(['state_successful', 'main_category_Comics', 'main_category_Crafts',
       'main_category_Dance', 'main_category_Design', 'main_category_Fashion',
       'main_category_Film & Video', 'main_category_Food',
       'main_category_Games', 'main_category_Journalism',
       'main_category_Music', 'main_category_Photography',
       'main_category_Publishing', 'main_category_Technology',
       'main_category_Theater', 'country_AU', 'country_BE', 'country_CA',
       'country_CH', 'country_DE', 'country_DK', 'country_ES', 'country_FR',
       'country_GB', 'country_HK', 'country_IE', 'country_IT', 'country_JP',
       'country_LU', 'country_MX', 'country_N,0"', 'country_NL', 'country_NO',
       'country_NZ', 'country_SE', 'country_SG', 'country_US', 'currency_CAD',
       'currency_CHF', 'currency_DKK', 'currency_EUR', 'currency_GBP',
       'currency_HKD', 'currency_JPY', 'currency_MXN', 'currency_NOK',
       'currency_NZD', 'currency_SEK', 'currency_SGD', 'currency_USD', 'ID',
  

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 296870 entries, 1031685482 to 1922604394
Data columns (total 67 columns):
state_successful              296870 non-null uint8
main_category_Comics          296870 non-null uint8
main_category_Crafts          296870 non-null uint8
main_category_Dance           296870 non-null uint8
main_category_Design          296870 non-null uint8
main_category_Fashion         296870 non-null uint8
main_category_Film & Video    296870 non-null uint8
main_category_Food            296870 non-null uint8
main_category_Games           296870 non-null uint8
main_category_Journalism      296870 non-null uint8
main_category_Music           296870 non-null uint8
main_category_Photography     296870 non-null uint8
main_category_Publishing      296870 non-null uint8
main_category_Technology      296870 non-null uint8
main_category_Theater         296870 non-null uint8
country_AU                    296870 non-null uint8
country_BE                    296870 non-nul

In [15]:
df.head()

,state_successful,main_category_Comics,main_category_Crafts,main_category_Dance,main_category_Design,main_category_Fashion,main_category_Film & Video,main_category_Food,main_category_Games,main_category_Journalism,...,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,campaign_length,pct_goal_achieved
idx,,,,,,,,,,,,,,,,,,,,,
1031685482,1,0,0,0,0,0,0,0,0,0,...,2017-12-06,6083.00,successful,133,US,1054.0,6083.00,2000.0,27,304.2
1176736360,1,0,0,0,0,0,0,0,0,0,...,2017-11-30,11169.56,successful,177,US,650.0,11169.56,10000.0,33,111.7
1894703097,1,0,0,0,0,0,0,0,0,0,...,2017-11-28,25417.20,successful,168,DE,0.0,30615.02,30112.5,35,101.7
311863059,1,0,0,0,0,0,0,0,0,0,...,2017-12-09,1743.00,successful,39,US,0.0,1743.00,1000.0,24,174.3
2076121120,0,0,0,0,0,0,0,1,0,0,...,2017-11-03,1.00,failed,1,US,0.0,1.00,200000.0,60,0.0


In [16]:
try:
    df.to_sql("kickstarter_data", engine, chunksize=20000, if_exists='replace')
except Exception as e:
    print(e)
    pass

# Temporary fix:
# df.to_pickle('data/kickstarter_data.pkl')
# df.to_csv('data/kickstarter_data.csv')

(psycopg2.OperationalError) SSL SYSCALL error: EOF detected
 (Background on this error at: http://sqlalche.me/e/e3q8)


## State of Success [this should now be covered in EDA]

In [17]:
# df_state = df.groupby(["state"]).count().sort_values(['name'], ascending=[False])
# df_state = pd.DataFrame(df_state, columns={'name'})
# print(df_state.shape)
# df_state

In [18]:
# df_state = df.groupby(["state"]).sum().sort_values(['usd pledged'], ascending=[False])
# df_state = pd.DataFrame(df_state, columns={'usd pledged', 'usd_pledged_real','usd_goal_real'})
# print(df_state.shape)
# df_state

In [19]:
# df_country = df.groupby(["country"]).count().sort_values(['name'], ascending=[False])
# df_country = pd.DataFrame(df_country, columns={'name'})
# print(df_country.shape)
# df_country

In [20]:
# df_country = df.groupby(["country"]).sum().sort_values(['usd pledged'], ascending=[False])
# df_country = pd.DataFrame(df_country, columns={'usd pledged', 'usd_pledged_real','usd_goal_real'})
# print(df_country.shape)
# df_country

In [21]:
# df_currency = df.groupby(["currency"]).count().sort_values(['name'], ascending=[False])
# df_currency = pd.DataFrame(df_currency, columns={'name'})
# print(df_currency.shape)
# df_currency

In [22]:
# df_currency = df.groupby(["currency"]).sum().sort_values(['usd pledged'], ascending=[False])
# df_currency = pd.DataFrame(df_currency, columns={'usd pledged', 'usd_pledged_real','usd_goal_real'})
# print(df_currency.shape)
# df_currency

In [23]:
# df_main_category = df.groupby(["main_category"]).count().sort_values(['name'], ascending=[False])
# df_main_category = pd.DataFrame(df_main_category, columns={'name'})
# print(df_main_category.shape)
# df_main_category

In [24]:
# df_main_category = df.groupby(["main_category"]).sum().sort_values(['usd pledged'], ascending=[False])
# df_main_category = pd.DataFrame(df_main_category, columns={'usd pledged', 'usd_pledged_real','usd_goal_real'})
# print(df_main_category.shape)
# df_main_category

In [25]:
# df_category = df.groupby(["category"]).count().sort_values(['name'], ascending=[False])
# print(df_category.shape)
# df_category

In [26]:
# df_category = df.groupby(["category"]).sum().sort_values(['usd pledged'], ascending=[False])
# df_category = pd.DataFrame(df_category, columns={'usd pledged', 'usd_pledged_real','usd_goal_real'})
# print(df_category.shape)
# df_category

In [27]:
# df2 = pd.read_csv('data/Kickstarter_2018-01-12T10_20_09_196Z/Kickstarter048.csv')
# df2.head()